# PCE vs NGD vs CasADi: Stability Comparison

**Cost function:** $J(x) = (x - 5)^2 + 0.1 x^2 + 100 \cdot \mathbf{1}_{(2,3)}(x) + 100 \cdot \mathbf{1}_{(6,8)}(x)$

**Optimal:** $x^* = \frac{50}{11} \approx 4.545$, $J^* = 2.27$

**Key insight:** PCE with best-so-far tracking is stable and reaches global optimum, while CasADi (with smooth barrier) gets stuck in local minimum.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from ipywidgets import interact, IntSlider
from enum import Enum
import casadi as ca

np.random.seed(42)

## Cost Functions

In [2]:
def quadratic_state_cost(x, x_star=5.0):
    return (x - x_star) ** 2

def control_cost(x, R=0.1):
    return R * x ** 2

def barrier_cost(x, barrier_val=100.0):
    """Indicator: large cost for x in (2,3) and (6,8)"""
    b1 = np.where((x > 2) & (x < 3), barrier_val, 0.0)
    b2 = np.where((x > 6) & (x < 8), barrier_val, 0.0)
    return b1 + b2

def total_cost(x, x_star=5.0, R=0.1):
    return quadratic_state_cost(x, x_star) + control_cost(x, R) + barrier_cost(x)

X_STAR = 50 / 11
OPTIMAL_COST = quadratic_state_cost(np.array([X_STAR]))[0] + control_cost(np.array([X_STAR]))[0]
print(f"Optimal x* = {X_STAR:.4f}, J* = {OPTIMAL_COST:.4f}")

Optimal x* = 4.5455, J* = 2.2727


## Covariance Scheduling

In [3]:
class CovarianceSchedule(Enum):
    CONSTANT = 0
    LINEAR = 1
    EXPONENTIAL = 2
    COSINE = 3

def compute_covariance_scale(iteration, n_iterations, schedule,
    cov_scale_initial=1.0, cov_scale_final=0.01, cov_decay_rate=0.95):
    t = float(iteration - 1)
    T = float(n_iterations)
    if schedule == CovarianceSchedule.CONSTANT: return cov_scale_initial
    elif schedule == CovarianceSchedule.LINEAR: return cov_scale_initial + (cov_scale_final - cov_scale_initial) * (t / T)
    elif schedule == CovarianceSchedule.EXPONENTIAL: return max(cov_scale_final, cov_scale_initial * (cov_decay_rate ** t))
    elif schedule == CovarianceSchedule.COSINE: return cov_scale_final + 0.5 * (cov_scale_initial - cov_scale_final) * (1.0 + np.cos(np.pi * t / T))
    return cov_scale_initial

## PCE Algorithm (with best-so-far tracking for stability)

In [4]:
def run_pce_with_history(cost_fn, y_init=0.0, sigma_init=2.0, n_samples=100, n_iterations=50,
    elite_ratio=0.1, ema_alpha=0.5, gamma=1.0, temperature=1.5, temperature_final=0.05,
    cov_schedule=CovarianceSchedule.COSINE, cov_scale_initial=1.0, cov_scale_final=0.01, cov_decay_rate=0.95):
    
    y = y_init
    y_best = y_init  # KEY: Track best solution found
    cost_best = cost_fn(np.array([y_init]))[0]
    n_elites = max(1, int(n_samples * elite_ratio))
    
    history = {'y': [y], 'y_best': [y_best], 'sigma': [sigma_init * cov_scale_initial],
               'cost': [cost_best], 'cost_best': [cost_best],
               'temperature': [temperature], 'samples': [], 'sample_costs': [],
               'elite_indices': [], 'elite_weights': [], 'weighted_y': []}
    
    for iteration in range(1, n_iterations + 1):
        progress = (iteration - 1) / max(1, n_iterations - 1)
        current_temp = temperature * (temperature_final / temperature) ** progress
        cov_scale = compute_covariance_scale(iteration, n_iterations, cov_schedule,
            cov_scale_initial, cov_scale_final, cov_decay_rate)
        effective_sigma = sigma_init * cov_scale
        
        # KEY FIX 1: Sample around BEST solution (not current y)
        epsilon = effective_sigma * np.random.randn(n_samples)
        samples = y_best + epsilon
        costs = cost_fn(samples)
        elite_indices = np.argsort(costs)[:n_elites]
        
        weights = np.zeros(n_samples)
        for m in elite_indices:
            weights[m] = np.exp((-gamma * costs[m]) / current_temp)
        weights /= (np.sum(weights[elite_indices]) + 1e-10)
        x_weighted = sum(weights[m] * samples[m] for m in elite_indices)
        
        history['samples'].append(samples.copy())
        history['sample_costs'].append(costs.copy())
        history['elite_indices'].append(elite_indices.copy())
        history['elite_weights'].append(weights[elite_indices].copy())
        history['weighted_y'].append(x_weighted)
        
        # Compute candidate
        y_new = (1 - ema_alpha) * y_best + ema_alpha * x_weighted
        cost_new = cost_fn(np.array([y_new]))[0]
        
        # KEY FIX 2: Only update y_best if improvement (monotonic!)
        if cost_new < cost_best:
            y_best = y_new
            cost_best = cost_new
        
        y = y_new  # Track raw trajectory
        history['y'].append(y)
        history['y_best'].append(y_best)
        history['sigma'].append(effective_sigma)
        history['cost'].append(cost_new)
        history['cost_best'].append(cost_best)
        history['temperature'].append(current_temp)
    
    return history

## NGD Algorithm

In [5]:
def run_ngd_with_history(cost_fn, y_init=0.0, sigma_init=2.0, n_samples=100, n_iterations=50,
    learning_rate=0.1, temperature=1.0, cov_schedule=CovarianceSchedule.COSINE,
    cov_scale_initial=1.0, cov_scale_final=0.01, cov_decay_rate=0.95):
    
    y = y_init
    prev_cost = cost_fn(np.array([y]))[0]
    
    history = {'y': [y], 'sigma': [sigma_init * cov_scale_initial], 'cost': [prev_cost],
               'temperature': [temperature], 'samples': [], 'sample_costs': [], 'gradient': [], 'epsilon': []}
    
    for iteration in range(1, n_iterations + 1):
        cov_scale = compute_covariance_scale(iteration, n_iterations, cov_schedule,
            cov_scale_initial, cov_scale_final, cov_decay_rate)
        effective_sigma = sigma_init * cov_scale
        
        epsilon = effective_sigma * np.random.randn(n_samples)
        samples = y + epsilon
        costs = cost_fn(samples)
        natural_gradient = np.mean((costs / temperature) * epsilon)
        
        history['samples'].append(samples.copy())
        history['sample_costs'].append(costs.copy())
        history['gradient'].append(natural_gradient)
        history['epsilon'].append(epsilon.copy())
        
        y = y - learning_rate * natural_gradient
        history['y'].append(y)
        history['sigma'].append(effective_sigma)
        history['cost'].append(cost_fn(np.array([y]))[0])
        history['temperature'].append(temperature)
    
    return history

## CasADi Solver (with smooth barrier approximation k=2)

In [6]:
def run_casadi_with_history(y_init=0.0, x_star=5.0, R=0.1, k=2):
    """CasADi with smooth sigmoid barrier approximation.
    Lower k = smoother = more artificial local minima!"""
    x = ca.SX.sym('x')
    base_cost = (x - x_star)**2 + R * x**2
    barrier1 = 100.0 * (1 / (1 + ca.exp(-k*(x - 2)))) * (1 / (1 + ca.exp(k*(x - 3))))
    barrier2 = 100.0 * (1 / (1 + ca.exp(-k*(x - 6)))) * (1 / (1 + ca.exp(k*(x - 8))))
    J = base_cost + barrier1 + barrier2
    grad_J = ca.gradient(J, x)
    
    cost_fn = ca.Function('J', [x], [J])
    grad_fn = ca.Function('dJ', [x], [grad_J])
    
    y = y_init
    history = {'y': [y], 'cost': [float(cost_fn(y))], 
               'true_cost': [total_cost(np.array([y]))[0]]}
    
    lr = 0.3
    for i in range(100):
        g = float(grad_fn(y))
        y_new = y - lr * g
        history['y'].append(y_new)
        history['cost'].append(float(cost_fn(y_new)))
        history['true_cost'].append(total_cost(np.array([y_new]))[0])
        
        if abs(g) < 1e-8:
            break
        y = y_new
        lr *= 0.98
    
    return {'y': history['y'], 'cost': history['cost'], 'true_cost': history['true_cost'],
            'x_opt': history['y'][-1], 'f_opt': history['true_cost'][-1]}

## Run All Algorithms

In [7]:
Y_INIT = 9.5  # Start from local minimum basin (for CasADi k=2)
SIGMA = 3.0
N_SAMPLES = 100
N_ITERATIONS = 50
COV_SCHEDULE = CovarianceSchedule.COSINE

np.random.seed(42)
pce_history = run_pce_with_history(cost_fn=total_cost, y_init=Y_INIT, sigma_init=SIGMA,
    n_samples=N_SAMPLES, n_iterations=N_ITERATIONS, cov_schedule=COV_SCHEDULE)

np.random.seed(42)
ngd_history = run_ngd_with_history(cost_fn=total_cost, y_init=Y_INIT, sigma_init=SIGMA,
    n_samples=N_SAMPLES, n_iterations=N_ITERATIONS, cov_schedule=COV_SCHEDULE)

casadi_history = run_casadi_with_history(y_init=Y_INIT, k=2)

print(f"Optimal: x*={X_STAR:.4f}, J*={OPTIMAL_COST:.4f}")
print(f"PCE (best-so-far): y_best={pce_history['y_best'][-1]:.4f}, cost_best={pce_history['cost_best'][-1]:.4f}")
print(f"NGD: y={ngd_history['y'][-1]:.4f}, cost={ngd_history['cost'][-1]:.4f}")
print(f"CasADi (k=2): y={casadi_history['x_opt']:.4f}, TRUE cost={casadi_history['f_opt']:.4f}")

Optimal: x*=4.5455, J*=2.2727
PCE (best-so-far): y_best=4.5457, cost_best=2.2727
NGD: y=1.3988, cost=13.1642
CasADi (k=2): y=9.4049, TRUE cost=28.2482


## Interactive Visualization (3×3 Layout)

In [8]:
def plot_iteration(iteration: int):
    casadi_iter = min(iteration, len(casadi_history['y']) - 1)
    
    fig = plt.figure(figsize=(16, 12))
    gs = GridSpec(3, 3, figure=fig, height_ratios=[1.2, 1, 1])
    
    x_plot = np.linspace(-2, 12, 500)
    cost_curve = total_cost(x_plot)
    
    # Smooth cost for CasADi visualization
    def smooth_cost(x_arr, k=2):
        return np.array([(xv - 5)**2 + 0.1 * xv**2 + 
                 100.0 / (1 + np.exp(-k*(xv - 2))) / (1 + np.exp(k*(xv - 3))) +
                 100.0 / (1 + np.exp(-k*(xv - 6))) / (1 + np.exp(k*(xv - 8)))
                 for xv in x_arr])
    
    # TOP LEFT: PCE
    ax = fig.add_subplot(gs[0, 0])
    ax.plot(x_plot, cost_curve, 'k-', lw=2, label='J(x)')
    ax.axvline(X_STAR, color='red', ls='--', lw=2, alpha=0.7, label=f'x*={X_STAR:.2f}')
    ax.axvspan(2, 3, alpha=0.3, color='orange', label='Barriers')
    ax.axvspan(6, 8, alpha=0.3, color='orange')
    if iteration < len(pce_history['samples']):
        samples = pce_history['samples'][iteration]
        costs = pce_history['sample_costs'][iteration]
        elite_idx = pce_history['elite_indices'][iteration]
        elite_weights = pce_history['elite_weights'][iteration]
        ax.scatter(samples, costs, c='lightgreen', s=40, alpha=0.6, edgecolors='gray', linewidths=0.5)
        sizes = 50 + 300 * elite_weights / elite_weights.max()
        ax.scatter(samples[elite_idx], costs[elite_idx], c='green', s=sizes, edgecolors='darkgreen', linewidths=1.5, alpha=0.8)
    # Show both raw and best trajectories
    y_hist = pce_history['y'][:iteration+2]
    y_best_hist = pce_history['y_best'][:iteration+2]
    cost_hist = pce_history['cost'][:iteration+2]
    cost_best_hist = pce_history['cost_best'][:iteration+2]
    ax.plot(y_hist, cost_hist, 'lightgreen', lw=1, alpha=0.4)
    ax.plot(y_best_hist, cost_best_hist, 'g-', lw=2, alpha=0.8)
    ax.scatter([pce_history['y_best'][iteration]], [pce_history['cost_best'][iteration]], 
               c='green', s=200, marker='*', edgecolors='darkgreen', linewidths=2)
    ax.annotate(f'Iter {iteration}\ny_best = {pce_history["y_best"][iteration]:.3f}\nCost = {pce_history["cost_best"][iteration]:.2f}', 
                xy=(0.02, 0.98), xycoords='axes fraction', fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
    ax.set_xlabel('x'); ax.set_ylabel('J(x)')
    ax.set_title('PCE (best-so-far)', fontsize=14, fontweight='bold', color='green')
    ax.legend(loc='upper right', fontsize=7); ax.set_xlim(-2, 12); ax.set_ylim(0, 50); ax.grid(True, alpha=0.3)
    
    # TOP MIDDLE: NGD
    ax = fig.add_subplot(gs[0, 1])
    ax.plot(x_plot, cost_curve, 'k-', lw=2, label='J(x)')
    ax.axvline(X_STAR, color='red', ls='--', lw=2, alpha=0.7, label=f'x*={X_STAR:.2f}')
    ax.axvspan(2, 3, alpha=0.3, color='orange', label='Barriers')
    ax.axvspan(6, 8, alpha=0.3, color='orange')
    if iteration < len(ngd_history['samples']):
        samples = ngd_history['samples'][iteration]
        costs = ngd_history['sample_costs'][iteration]
        epsilon = ngd_history['epsilon'][iteration]
        ax.scatter(samples, costs, c='lightblue', s=40, alpha=0.6, edgecolors='gray', linewidths=0.5)
        contributions = costs * epsilon
        high_mask = np.abs(contributions) > np.percentile(np.abs(contributions), 90)
        ax.scatter(samples[high_mask], costs[high_mask], c='blue', s=80, edgecolors='darkblue', linewidths=1.5, alpha=0.8)
    y_hist = ngd_history['y'][:iteration+2]
    cost_hist = ngd_history['cost'][:iteration+2]
    ax.plot(y_hist, cost_hist, 'b-', lw=2, alpha=0.5)
    ax.scatter([ngd_history['y'][iteration]], [ngd_history['cost'][iteration]], 
               c='blue', s=200, marker='*', edgecolors='darkblue', linewidths=2)
    ax.annotate(f'Iter {iteration}\ny = {ngd_history["y"][iteration]:.3f}\nCost = {ngd_history["cost"][iteration]:.2f}', 
                xy=(0.02, 0.98), xycoords='axes fraction', fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))
    ax.set_xlabel('x'); ax.set_ylabel('J(x)')
    ax.set_title('NGD', fontsize=14, fontweight='bold', color='blue')
    ax.legend(loc='upper right', fontsize=7); ax.set_xlim(-2, 12); ax.set_ylim(0, 50); ax.grid(True, alpha=0.3)
    
    # TOP RIGHT: CasADi
    ax = fig.add_subplot(gs[0, 2])
    ax.plot(x_plot, cost_curve, 'k-', lw=2, label='True J(x)')
    ax.plot(x_plot, smooth_cost(x_plot), 'purple', lw=1.5, ls='--', alpha=0.5, label='Smooth J (k=2)')
    ax.axvline(X_STAR, color='red', ls='--', lw=2, alpha=0.7)
    ax.axvspan(2, 3, alpha=0.3, color='orange')
    ax.axvspan(6, 8, alpha=0.3, color='orange')
    y_hist_ca = casadi_history['y'][:casadi_iter+2]
    cost_hist_ca = casadi_history['true_cost'][:casadi_iter+2]
    ax.plot(y_hist_ca, cost_hist_ca, 'purple', lw=2, alpha=0.5)
    ax.scatter(y_hist_ca[:-1], cost_hist_ca[:-1], c='purple', s=60, alpha=0.4, marker='o')
    ax.scatter([casadi_history['y'][casadi_iter]], [casadi_history['true_cost'][casadi_iter]], 
               c='purple', s=200, marker='*', edgecolors='darkmagenta', linewidths=2)
    ax.annotate(f'Iter {casadi_iter}\ny = {casadi_history["y"][casadi_iter]:.3f}\nTRUE Cost = {casadi_history["true_cost"][casadi_iter]:.2f}', 
                xy=(0.02, 0.98), xycoords='axes fraction', fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lavender', alpha=0.7))
    ax.set_xlabel('x'); ax.set_ylabel('J(x)')
    ax.set_title('CasADi k=2 (smooth barrier)', fontsize=14, fontweight='bold', color='purple')
    ax.legend(loc='upper right', fontsize=7); ax.set_xlim(-2, 12); ax.set_ylim(0, 50); ax.grid(True, alpha=0.3)
    
    # MIDDLE ROW: y trajectory
    ax = fig.add_subplot(gs[1, 0])
    ax.plot(pce_history['y'], 'lightgreen', lw=1, alpha=0.4, label='Raw y')
    ax.plot(pce_history['y_best'], 'g-', lw=2, label='y_best')
    ax.axhline(X_STAR, color='red', ls='--', lw=1.5, alpha=0.7, label='x*')
    ax.axhspan(2, 3, alpha=0.15, color='orange'); ax.axhspan(6, 8, alpha=0.15, color='orange')
    ax.scatter([iteration], [pce_history['y_best'][iteration]], c='green', s=150, zorder=10, marker='*')
    ax.axvline(iteration, color='gray', ls='--', lw=1, alpha=0.5)
    ax.set_xlabel('Iteration'); ax.set_ylabel('y')
    ax.set_title('PCE: y_best never regresses', fontsize=12, fontweight='bold', color='green')
    ax.legend(loc='upper right', fontsize=8); ax.grid(True, alpha=0.3)
    
    ax = fig.add_subplot(gs[1, 1])
    ax.plot(ngd_history['y'], 'b-', lw=2)
    ax.axhline(X_STAR, color='red', ls='--', lw=1.5, alpha=0.7, label='x*')
    ax.axhspan(2, 3, alpha=0.15, color='orange'); ax.axhspan(6, 8, alpha=0.15, color='orange')
    ax.scatter([iteration], [ngd_history['y'][iteration]], c='blue', s=150, zorder=10, marker='*')
    ax.axvline(iteration, color='gray', ls='--', lw=1, alpha=0.5)
    ax.set_xlabel('Iteration'); ax.set_ylabel('y')
    ax.set_title('NGD: y Trajectory', fontsize=12, fontweight='bold', color='blue')
    ax.legend(loc='upper right', fontsize=8); ax.grid(True, alpha=0.3)
    
    ax = fig.add_subplot(gs[1, 2])
    ax.plot(casadi_history['y'], 'purple', lw=2, marker='o', markersize=4)
    ax.axhline(X_STAR, color='red', ls='--', lw=1.5, alpha=0.7, label='x*')
    ax.axhspan(2, 3, alpha=0.15, color='orange'); ax.axhspan(6, 8, alpha=0.15, color='orange')
    ax.scatter([casadi_iter], [casadi_history['y'][casadi_iter]], c='purple', s=150, zorder=10, marker='*')
    ax.axvline(casadi_iter, color='gray', ls='--', lw=1, alpha=0.5)
    ax.set_xlabel('Iteration'); ax.set_ylabel('y')
    ax.set_title('CasADi: stuck in local min', fontsize=12, fontweight='bold', color='purple')
    ax.legend(loc='upper right', fontsize=8); ax.grid(True, alpha=0.3)
    
    # BOTTOM ROW: Cost convergence
    ax = fig.add_subplot(gs[2, 0])
    ax.plot(pce_history['cost'], 'lightgreen', lw=1, alpha=0.4, label='Raw cost')
    ax.plot(pce_history['cost_best'], 'g-', lw=2, label='cost_best')
    ax.axhline(OPTIMAL_COST, color='red', ls='--', lw=1.5, alpha=0.7, label=f'Opt={OPTIMAL_COST:.2f}')
    ax.scatter([iteration], [pce_history['cost_best'][iteration]], c='green', s=150, zorder=10, marker='*')
    ax.axvline(iteration, color='gray', ls='--', lw=1, alpha=0.5)
    ax.set_xlabel('Iteration'); ax.set_ylabel('Cost')
    ax.set_title('PCE: cost_best monotonic ↓', fontsize=12, fontweight='bold', color='green')
    ax.set_yscale('log'); ax.legend(loc='upper right', fontsize=8); ax.grid(True, alpha=0.3)
    
    ax = fig.add_subplot(gs[2, 1])
    ax.plot(ngd_history['cost'], 'b-', lw=2)
    ax.axhline(OPTIMAL_COST, color='red', ls='--', lw=1.5, alpha=0.7, label=f'Opt={OPTIMAL_COST:.2f}')
    ax.scatter([iteration], [ngd_history['cost'][iteration]], c='blue', s=150, zorder=10, marker='*')
    ax.axvline(iteration, color='gray', ls='--', lw=1, alpha=0.5)
    ax.set_xlabel('Iteration'); ax.set_ylabel('Cost')
    ax.set_title('NGD: Cost', fontsize=12, fontweight='bold', color='blue')
    ax.set_yscale('log'); ax.legend(loc='upper right', fontsize=8); ax.grid(True, alpha=0.3)
    
    ax = fig.add_subplot(gs[2, 2])
    ax.plot(casadi_history['true_cost'], 'purple', lw=2, marker='o', markersize=4)
    ax.axhline(OPTIMAL_COST, color='red', ls='--', lw=1.5, alpha=0.7, label=f'Opt={OPTIMAL_COST:.2f}')
    ax.scatter([casadi_iter], [casadi_history['true_cost'][casadi_iter]], c='purple', s=150, zorder=10, marker='*')
    ax.axvline(casadi_iter, color='gray', ls='--', lw=1, alpha=0.5)
    ax.set_xlabel('Iteration'); ax.set_ylabel('TRUE Cost')
    ax.set_title('CasADi: TRUE Cost', fontsize=12, fontweight='bold', color='purple')
    ax.set_yscale('log'); ax.legend(loc='upper right', fontsize=8); ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [9]:
interact(
    plot_iteration,
    iteration=IntSlider(min=0, max=N_ITERATIONS-1, step=1, value=0,
        description='Iteration:', continuous_update=False, style={'description_width': 'initial'})
);

interactive(children=(IntSlider(value=0, continuous_update=False, description='Iteration:', max=49, style=Slid…